# Tennis Elo Rating

Import standard packages

In [1]:
import pandas as pd
import numpy as np

In [2]:
utr_train = pd.read_csv('utr_train_UPDATED.csv')
utr_test = pd.read_csv('utr_test_UPDATED.csv')

In [3]:
utr_train.head()

,resultid,resultmonth,winnerid,loserid,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5
0,1,2019-05,57529,3765,7,7,0,0,0,6,5,0,0,0
1,2,2019-03,83218,3871,6,6,0,0,0,4,2,0,0,0
2,4,2019-11,4021,4487,7,5,7,0,0,5,7,5,0,0
3,5,2019-10,1984892,411593,6,6,0,0,0,3,3,0,0,0
4,7,2019-09,52294,224678,6,6,0,0,0,3,2,0,0,0


Sort matches chronologically and drop undeeded columns

In [4]:
#utr_train = utr_train[['resultmonth','winnerid','loserid']]
utr_train = utr_train.sort_values('resultmonth')
utr_train = utr_train.drop(columns=['resultmonth'])
utr_train

,resultid,winnerid,loserid,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5
420,578,11063,45027,6,6,0,0,0,1,0,0,0,0
2818,4026,847617,10788,1,6,6,0,0,6,3,4,0,0
2816,4022,11076,2555950,6,6,0,0,0,3,2,0,0,0
1593,2249,58235,79636,6,6,0,0,0,2,3,0,0,0
1589,2244,3498,83218,4,6,6,0,0,6,4,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
264,382,86294,233912,6,6,0,0,0,3,3,0,0,0
3886,5591,238409,153449,4,6,6,0,0,6,3,1,0,0
3903,5613,50083,191924,2,6,6,0,0,6,2,2,0,0
2668,3816,11640,42381,1,6,6,0,0,6,1,2,0,0


Create **player_list** variable that will keep track of/update a players elo after each inputed match.

Note: I assume everyone starts off with the same elo at 1500

In [5]:
player_list = (utr_train['winnerid'].append(utr_train['loserid'])).unique()

player_df = pd.DataFrame(player_list, columns=['id'])
player_df = player_df.assign(elo = 1500.0)

In [6]:
player_df

,id,elo
0,11063,1500.0
1,847617,1500.0
2,11076,1500.0
3,58235,1500.0
4,3498,1500.0
...,...,...
1141,34161,1500.0
1142,217710,1500.0
1143,25603,1500.0
1144,90999,1500.0


In [7]:
import math 

# Formula to calculate prob of Player 1 winning against player 2
# Derived from Arpad Elo chess algorithm: https://en.wikipedia.org/wiki/Elo_rating_system

def expectedValue(rating1, rating2): 
  
    return(1.0/(1.0 + 10.0 ** ((rating2 - rating1) / 400)))
  
    
  
 # Function to calculate/update Elo rating of both players
# K is a constant that changes degree of elo gain/loss per matchup
# d determines whether Player 1 or Player 2 wins 

def EloRating2(R1, R2, K=32): 
   
  
    # To calculate the Winning 
    # Probability of Player 2 
    E1 = expectedValue(R1, R2) 
  
    # To calculate the Winning 
    # Probability of Player 1 
    E2 = expectedValue(R2, R1) 
  
    # Updating the Elo Ratings 
    R1 = R1 + K * (1 - E1) 
    R2 = R2 + K * (0 - E2)       
   
    return(R1,R2) 
  
    
# Test:      
# Ra and Rb are current Elo ratings 
Ra = 1200
Rb = 1000
EloRating2(Ra, Rb) 
  



(1207.6880983472654, 992.3119016527346)

Large forloop through **the utr_train** dataframe
    -Compute each match from training set & update each players Elo in **player_df** dataframe after each match

In [8]:
for i in range(0,len(utr_train)):
    m_winner = utr_train.iloc[i,1]
    m_loser = utr_train.iloc[i,2]
    
    winner_df_index = player_df.id[player_df.id == m_winner].index[0]
    loser_df_index = player_df.id[player_df.id == m_loser].index[0]
    
    winner_rating = player_df.iloc[winner_df_index,1]
    loser_rating = player_df.iloc[loser_df_index,1]
    

    new_winner_rank, new_loser_rank = EloRating2(winner_rating, loser_rating)
    
    print(m_winner, m_loser)
    print(winner_rating, loser_rating)
    print(new_winner_rank, new_loser_rank)
    print('\n')
    
    # Update the rankings in player_df after each match
    
    player_df.at[winner_df_index,'elo'] = new_winner_rank
    player_df.at[loser_df_index,'elo'] = new_loser_rank


    

11063 45027
1500.0 1500.0
1516.0 1484.0


847617 10788
1500.0 1500.0
1516.0 1484.0


11076 2555950
1500.0 1500.0
1516.0 1484.0


58235 79636
1500.0 1500.0
1516.0 1484.0


3498 83218
1500.0 1500.0
1516.0 1484.0


3691 3469
1500.0 1500.0
1516.0 1484.0


3564 224142
1500.0 1500.0
1516.0 1484.0


52337 51245
1500.0 1500.0
1516.0 1484.0


3832 52294
1500.0 1500.0
1516.0 1484.0


10788 1609206
1484.0 1500.0
1500.736306793522 1483.263693206478


3469 3685
1484.0 1500.0
1500.736306793522 1483.263693206478


196974 3700
1500.0 1500.0
1516.0 1484.0


10819 10794
1500.0 1500.0
1516.0 1484.0


10246 57418
1500.0 1500.0
1516.0 1484.0


10292 10259
1500.0 1500.0
1516.0 1484.0


50083 2602428
1500.0 1500.0
1516.0 1484.0


10347 57418
1500.0 1484.0
1515.263693206478 1468.736306793522


3478 200425
1500.0 1500.0
1516.0 1484.0


235621 4193
1500.0 1500.0
1516.0 1484.0


235621 206781
1516.0 1500.0
1531.263693206478 1484.736306793522


4205 1740216
1500.0 1500.0
1516.0 1484.0


206796 200399
1500.0 1500.

10346 10788
1483.2975328274754 1484.7023986633528
1499.3622289375062 1468.637702553322


10340 10479
1516.0 1516.0
1532.0 1500.0


3478 4205
1545.2178322106142 1559.729526119354
1561.8857320584962 1543.061626271472


4560 198546
1516.132691196011 1516.736306793522
1532.1604886935777 1500.7085092959553


51547 11695
1502.1400287537112 1516.736306793522
1518.8118170175694 1500.064518529664


71334 28990
1483.9660918698307 1500.0
1500.703956876504 1483.2621349933268


53149 52873
1484.797712172541 1500.0
1501.497356951837 1483.300355220704


3734 51256
1482.5700862902465 1500.0
1499.3720907749946 1483.197995515252


10383 52345
1500.7988653402276 1484.0321509180872
1516.0273284527743 1468.8036878055404


11349 10383
1531.263693206478 1516.0273284527743
1546.5624821341482 1500.728539525104


3498 3959
1499.263693206478 1514.5295224224085
1515.9662585562367 1497.8269570726497


11134 11035
1486.8671774055624 1485.4358728782763
1502.801263769081 1469.5017865147577


3927 224142
1500.0 1469.5

4372 3499
1467.241762475684 1468.039375019288
1483.2784938262134 1452.0026436687585


4031 81226
1531.263693206478 1484.0
1545.1004474267124 1470.1632457797655


10479 50083
1519.5015642178942 1526.5200108716147
1535.8247316737193 1510.1968434157895


3723 1978582
1484.0 1467.9676502352856
1499.2622066060967 1452.705443629189


219786 10767
1528.4974352782892 1500.0
1543.1880150064 1485.3094202718892


10793 2492783
1516.736306793522 1484.736306793522
1531.2668052645465 1470.2058083224977


53374 3587
1561.512650939965 1530.1956610328193
1576.0743434583298 1515.6339685144544


65308 55856
1517.3725163550669 1500.0
1532.573148496723 1484.7993678583439


10344 10695
1530.7320269453455 1501.4634432678208
1545.387338290773 1486.8081319223934


3475 206794
1501.4720231738654 1516.0
1518.1406715696855 1499.3313516041799


11558 58272
1485.4695015289756 1500.0
1502.138265847991 1483.3312356809845


2602428 2555959
1485.4342469519336 1517.4358728782763
1502.9038227259457 1499.9662971042642


2

10819 10383
1531.1699351523825 1500.686764997088
1545.7697243354633 1486.0869758140072


200399 3832
1527.4427328985093 1529.858511780129
1543.5539818344957 1513.7472628441426


51547 10344
1547.6355313732756 1528.1024231103945
1562.7369450399613 1513.0010094437089


3587 4151
1498.2392471515598 1504.8982751899744
1514.5458691811145 1488.5916531604196


83218 3458
1461.7519084264209 1516.403648484623
1480.248159368712 1497.9073975423319


2093432 2466141
1484.0 1500.0
1500.736306793522 1483.263693206478


3867 2611704
1544.619968781001 1514.0829732575382
1559.2172982312866 1499.4856438072527


3659 3507
1537.1616084155344 1488.3772475887329
1550.9296545245058 1474.6092014797614


10823 10346
1500.0 1513.39815977542
1516.616702389297 1496.781457386123


10246 10383
1527.2163843951628 1486.0869758140072
1541.3311035747397 1471.9722566344303


71334 11348
1499.3374630459662 1500.123778008151
1515.3736741263476 1484.0875669277696


10281 10352
1566.4808527007983 1470.138265847991
1578.1544

73237 52886
1429.9758824146247 1489.1974861875565
1448.6770285372152 1470.496340064966


11095 50083
1470.0213974258195 1510.1968434157895
1487.8633426768747 1492.3548981647343


196974 3458
1559.1952281195297 1497.9073975423319
1572.4017344800689 1484.7008911817927


83218 1194009
1500.6407724605226 1491.0098074227346
1516.1973636968837 1475.4532161863735


28290 2035498
1500.0 1490.1272001375169
1515.5454631008602 1474.5817370366567


1969408 28399
1497.683220324771 1458.3966942660077
1511.8816806347247 1444.1982339560539


219786 11342
1551.5287922152854 1541.3373443031655
1567.059593260116 1525.806543258335


10759 10885
1531.9473112369076 1474.7711075088978
1545.337764644277 1461.3806541015283


10281 10765
1578.1544389159135 1530.5636025699584
1591.9764049193273 1516.7416365665447


3850 253894
1500.0 1516.0
1516.736306793522 1499.263693206478


4031 3943
1582.9764352688098 1500.0
1595.2262598423488 1487.750175426461


142720 1972277
1484.0 1500.0
1500.736306793522 1483.263693206

...

51289 2616345
1454.695669716874 1514.3519204477554
1473.416257477902 1495.6313326867273


67623 196974
1598.6850443221188 1568.835087713899
1613.313775322945 1554.2063567130726


200399 3564
1573.1960327823226 1513.1186488514686
1586.456615736733 1499.8580658970582


3878 52341
1554.749919474139 1569.4588067804182
1571.4268843736627 1552.7818418808945


1398935 57433
1500.0975404737515 1500.0
1516.093048569053 1484.0044919046984


11076 34144
1567.8470352038685 1457.9793974561505
1578.949609624129 1446.87682303589


4151 1398935
1456.6504013974059 1516.093048569053
1475.361435760121 1497.382014206338


1972277 153449
1470.1257832802648 1500.0
1487.49816128599 1482.627621994275


3884 67499
1505.3958420091853 1444.3282024585712
1518.612180526636 1431.1118639411206


54171 197507
1515.145088517463 1455.7516348347149
1528.4362546109808 1442.460468741197


806774 60718
1463.8062446485058 1457.4556404759662
1479.5138210844102 1441.7480640400618


51289 212742
1473.416257477902 1452.89824774

11541 2502217
1499.6481356319975 1533.3928885731582
1517.1972708391042 1515.8437533660515


54883 4193
1477.6025477782948 1482.5626446186552
1493.8309531620712 1466.3342392348789


50772 4334
1639.8136313748323 1500.0
1649.7014030877003 1490.112228287132


1863628 28080
1437.6630148672643 1534.2888176430388
1458.0015149315625 1513.9503175787406


177312 54171
1435.8546263668359 1528.4362546109808
1456.0200403319989 1508.2708406458178


53374 198546
1689.8655681218534 1560.9723496762795
1700.1879273528411 1550.6499904452917


77968 54171
1639.8566415789428 1508.2708406458178
1650.0709160776107 1498.0565661471499


2500037 185816
1506.6470745578645 1500.0
1522.3410028046294 1484.306071753235


27419 1157738
1479.4059766470507 1443.0755344728373
1493.7389694085587 1428.7425417113293


85223 2231281
1546.140918426444 1551.8757799517816
1562.4049945769586 1535.611703801267


11788 219811
1467.5792971409521 1500.0
1485.0680071809177 1482.5112899600344


124948 960543
1456.214763513142 1485.9

2554103 28836
1486.034203253845 1485.5019951042245
1502.009694181981 1469.5265041760883


4075 2525708
1492.6813561580218 1487.259704131694
1508.4317001194702 1471.5093601702456


10246 10346
1570.3100369440601 1526.60364166187
1584.3078334998665 1512.6058451060635


1194009 4169
1523.9498212415572 1470.6692569834383
1537.5152159099096 1457.103862315086


52346 10246
1532.530851869911 1584.3078334998665
1550.8977737240684 1565.940911645709


52833 53085
1478.373878085695 1500.0
1495.3685135850124 1483.0053645006826


378488 10473
1575.8590207698971 1628.839451272068
1594.2801226801053 1610.4183493618598


3761 224142
1556.7789618618556 1490.454174109875
1569.7611620573289 1477.4719739144018


182464 274770
1500.0 1500.0
1516.0 1484.0


3699 203845
1488.3467768270286 1470.101483952786
1503.5073215634368 1454.940939216378


73237 3993
1508.231856048733 1621.0002711551133
1529.2500402303726 1599.9820869734738


28298 273950
1533.2068032532652 1499.426224534597
1547.6560336905272 1484.9769

405461 28527
1592.396258562265 1442.8377745706298
1601.9049155179982 1433.3291176148966


28527 51259
1433.3291176148966 1464.287981016498
1450.7510645253592 1446.8660341060354


1570645 60718
1520.629597619231 1448.4572222068853
1533.3529326083753 1435.7338872177409


3982 45571
1566.7188670589692 1445.8204114731059
1577.3656299724291 1435.173648559646


1978582 4073
1514.7642924364634 1571.2122739737208
1533.34118411429 1552.6353822958943


207039 1609083
1415.096007890445 1500.0
1434.929964196578 1480.1660436938669


3478 52886
1679.916696763816 1563.8806853553024
1690.7632394531176 1553.0341426660007


51289 95506
1493.3514569505035 1516.478578999336
1510.4149300199415 1499.415105929898


1398935 87188
1525.3719440090924 1484.7373602174684
1539.5091380886222 1470.6001661379387


224678 77968
1479.432882696504 1634.0452866749604
1502.1175038673427 1611.3606655041217


58915 2566922
1496.8134443381327 1513.8051364714238
1513.5953174166636 1497.0232633928929


55282 58977
1490.4013308

10794 2583619
1449.425385830995 1500.0
1467.7381208364295 1481.6872649945656


86294 51804
1485.6104018722747 1500.0
1502.2726887173528 1483.3377131549219


247320 224678
1648.8732943070374 1502.1175038673427
1658.4901270662865 1492.5006711080937


1988251 1083703
1491.3859331444755 1509.934070710785
1508.239295891755 1493.0807079635053


1948598 74092
1487.8908656004592 1531.1029894498963
1505.870659470649 1513.1231955797066


4348 2070519
1507.2078712587956 1518.8713536613122
1523.744792792121 1502.334432127987


96338 4487
1549.831326874332 1496.5874013136106
1563.3983697771414 1483.0203584108012


50817 5600
1484.1727305130275 1500.0
1500.9010994373357 1483.2716310756919


53125 74171
1438.681927612653 1493.1435675710068
1457.1696364842123 1474.6558586994474


247320 4622
1658.4901270662865 1492.5637691006682
1667.3813436181372 1483.6725525488175


3882 55965
1501.0193862276271 1470.1816641528776
1515.6029741906561 1455.5980761898486


50772 2536184
1642.9220708713983 1461.49201715

In [9]:
player_df

,id,elo
0,11063,1529.688787
1,847617,1468.706256
2,11076,1588.469766
3,58235,1548.220625
4,3498,1499.984660
...,...,...
1141,34161,1483.858080
1142,217710,1482.699653
1143,25603,1483.366618
1144,90999,1483.112875


## Calculating Elo Prob

In [10]:
# Adding a prob column to player df to test
utr_train = utr_train.assign(prob = 0.0)
utr_train = utr_train.reset_index()
utr_train

,index,resultid,winnerid,loserid,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5,prob
0,420,578,11063,45027,6,6,0,0,0,1,0,0,0,0,0.0
1,2818,4026,847617,10788,1,6,6,0,0,6,3,4,0,0,0.0
2,2816,4022,11076,2555950,6,6,0,0,0,3,2,0,0,0,0.0
3,1593,2249,58235,79636,6,6,0,0,0,2,3,0,0,0,0.0
4,1589,2244,3498,83218,4,6,6,0,0,6,4,4,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4547,264,382,86294,233912,6,6,0,0,0,3,3,0,0,0,0.0
4548,3886,5591,238409,153449,4,6,6,0,0,6,3,1,0,0,0.0
4549,3903,5613,50083,191924,2,6,6,0,0,6,2,2,0,0,0.0
4550,2668,3816,11640,42381,1,6,6,0,0,6,1,2,0,0,0.0


In [11]:
for i in range(0,len(utr_train)):
    m_winner = utr_train.iloc[i,2]
    m_loser = utr_train.iloc[i,3]
    
    winner_df_index = player_df.id[player_df.id == m_winner].index[0]
    loser_df_index = player_df.id[player_df.id == m_loser].index[0]
    
    winner_rating = player_df.iloc[winner_df_index,1]
    loser_rating = player_df.iloc[loser_df_index,1]
    
    current_prob = expectedValue(winner_rating, loser_rating)
    utr_train.at[i,'prob'] = current_prob
    
    print(m_winner,m_loser)
    print(winner_df_index,loser_df_index)
    print(winner_rating,loser_rating)
    print(current_prob)
    print('\n')


11063 45027
0 70
1529.6887873246224 1582.2730665797417
0.42489771468644044


847617 10788
1 9
1468.706256370886 1463.6472703889215
0.5072799515743568


11076 2555950
2 177
1588.4697656000594 1506.1063478000608
0.6163589003480863


58235 79636
3 31
1548.220624511203 1502.8724743948012
0.5648931428781356


3498 83218
4 100
1499.9846602994428 1650.9100075797844
0.29550486257880626


3691 3469
5 10
1536.5394746436443 1577.4410682590449
0.4414082995136355


3564 224142
6 149
1484.583256861686 1477.4719739144018
0.5102325299325895


52337 51245
7 459
1432.0889928218917 1445.6248072531514
0.48053024669662203


3832 52294
8 85
1550.812357134009 1576.8903504861128
0.4625410694781406


10788 1609206
9 555
1463.6472703889215 1457.127807161923
0.5093811607428198


3469 3685
10 700
1577.4410682590449 1476.3940667506135
0.6414523413246986


196974 3700
11 216
1608.7106127628465 1565.434607362588
0.5619590817508622


10819 10794
12 322
1597.1980660692589 1467.7381208364295
0.6781387476253319


10246 

204410 53840
88 132
1546.8860888516945 1630.844985255219
0.3814717050331409


10473 52346
27 131
1610.4183493618598 1550.8977737240684
0.5848287480229472


53840 3734
132 143
1630.844985255219 1512.4008170996092
0.6641446564792571


11076 65308
2 161
1588.4697656000594 1522.8016162840065
0.5933945586225323


4445 3721
133 787
1541.1005793374643 1414.158269081021
0.6749673494859294


268261 52346
134 131
1563.3239496831973 1550.8977737240684
0.5178750836296921


3587 3761
50 178
1493.1570563214464 1570.0887184484563
0.39106107012044516


28145 50083
57 15
1527.740914243151 1492.9051681524718
0.5499653444775527


204410 4726
88 446
1546.8860888516945 1571.3384286741352
0.46486824085619816


54351 204410
110 88
1466.8863970465143 1546.8860888516945
0.38686360066641284


51547 52345
135 124
1574.2825036354743 1434.4976269501085
0.6909717905104481


1244046 98984
136 788
1499.2975328274754 1426.9635867740467
0.6026185274596622


10281 10346
137 141
1600.6649497123542 1544.7527872358155
0.57

3654 94283
203 68
1545.0203163153838 1665.200025177391
0.33363054736973957


11076 58244
2 786
1588.4697656000594 1470.1141340458285
0.6640309645035599


57418 28900
79 162
1520.9389009221889 1511.8077977315243
0.5131376891523485


151401 60157
204 94
1513.2937971716924 1462.4992742795157
0.5725828023637759


54309 28491
205 804
1512.0454301880861 1484.0
0.5402731841412604


10322 2231281
80 47
1549.1338902854436 1519.2806402390474
0.5428568610451535


2616345 60718
206 389
1504.7573620279916 1440.0068671577167
0.5921193979253639


259120 51304
207 296
1436.3691356115135 1516.7887189286619
0.3862904236018242


10346 55858
141 805
1544.7527872358155 1468.6250324058294
0.6078363800817511


2616345 3711
206 231
1504.7573620279916 1507.791780588989
0.4956332316893878


203816 4169
208 240
1527.836240820635 1457.103862315086
0.6004086965496771


76892 47233
209 806
1466.8888173168893 1467.3664594112004
0.49931261820420336


3698 3718
21 466
1481.9219116199433 1463.5176446419298
0.5264611238


10378 11042
22 69
1619.3321933481923 1514.392490988625
0.6465895297609705


203816 2596786
208 197
1527.836240820635 1488.8509236160166
0.5558700948037376


3961 49953
182 78
1589.5663175881707 1546.2415929809185
0.562028116497231


3619 3778
264 823
1523.362235826676 1484.1227547593207
0.5562312694452161


10850 52003
241 588
1525.0064982491572 1421.630973703058
0.6445292766937348


10283 10788
116 9
1485.45615177099 1463.6472703889215
0.5313443463814104


10788 53395
9 784
1463.6472703889215 1465.3514195314135
0.49754755190995004


2118860 2611278
270 824
1515.900582530439 1468.0211615156554
0.5684711270591666


259120 10766
207 139
1436.3691356115135 1501.2515415088806
0.4076972234373491


70649 10322
42 80
1658.0006640141014 1549.1338902854436
0.6517379898244889


1744266 3460
234 173
1529.9782050655747 1464.2017595482916
0.5935449629621239


70414 74716
271 504
1480.7384808727293 1471.5143001597764
0.5132715450509772


10765 2595933
272 825
1559.504057751005 1385.620241000417
0.7

4515 2611704
171 97
1634.5705193617312 1510.2975061666473
0.6715873007352001


206783 51288
313 317
1563.613688557894 1496.0104570941858
0.596079382385347


10310 268262
314 76
1509.0515408580209 1477.9483751248067
0.544641860887727


3700 206796
216 20
1565.434607362588 1574.5233315732198
0.4869232571914639


10274 57418
82 79
1555.9448798695119 1520.9389009221889
0.5502078709042187


2526148 256513
315 481
1513.740367793247 1493.2593720158723
0.5294404283968783


2231281 268262
47 76
1519.2806402390474 1477.9483751248067
0.559202886630823


260593 3488
286 186
1551.5949611530366 1507.6658944748465
0.5628842641217526


162815 802298
316 373
1525.6807325370312 1490.0883611368085
0.5510431059523795


77442 11348
227 243
1542.2689239048782 1536.2324877033968
0.5086862560000032


224134 3486
109 571
1522.810706473156 1484.927074189107
0.5543038888199309


65308 256580
161 835
1522.8016162840065 1485.3956591248764
0.5536244667131618


51288 3597
317 340
1496.0104570941858 1515.341563773159

...

3495 3701
126 198
1507.5337348330117 1471.2171641705515
0.5520742301084386


2005359 3488
279 186
1593.8730672829656 1507.6658944748465
0.6215772818000153


3466 253923
194 463
1510.039229346356 1439.7920387311701
0.5997384259354284


192639 2600371
662 425
1530.5933345526703 1498.5244940297014
0.5460201547878692


4686 4031
122 147
1585.4656633524557 1550.0265404219092
0.5508248522649467


1710702 59382
703 687
1500.9941407851788 1497.8066784046266
0.504586998410727


57248 268917
704 1061
1499.2201192769824 1484.0
0.5218895117462433


45027 11558
70 213
1582.2730665797417 1598.5923437760448
0.47653192846376524


52598 2115126
705 506
1501.1171818449025 1481.5323654925878
0.5281550015067386


4073 3685
473 700
1580.6223903706323 1476.3940667506135
0.6456532086156043


2500037 80810
479 519
1522.1074428342715 1476.7320277853105
0.5649317189437422


4372 53449
211 158
1507.8863485408053 1552.0575931819903
0.4367727567228354


200425 207438
23 168
1608.1135344797276 1482.8054159325518
0.

10537 11703
623 1073
1534.2211002769682 1484.8786929979217
0.5705358476676462


1978582 49884
278 419
1570.5478633597309 1495.9971392340797
0.6056703114412146


10347 10303
16 678
1567.9655019266418 1477.437780060531
0.6274094955615327


1554636 56797
599 644
1479.2281830407292 1482.94722903701
0.49464806698977865


2095984 2103686
460 440
1529.2917825232719 1472.8427109543593
0.580529392923006


10751 57418
167 79
1453.3461136515248 1520.9389009221889
0.4039350931028159


57418 53243
79 937
1520.9389009221889 1469.710319719106
0.5731941877927232


91084 64324
447 537
1595.6246688827584 1533.9910453912244
0.5877790588992795


3659 66480
49 323
1506.3329842080473 1442.888122613037
0.5903029707461371


196974 53255
11 165
1608.7106127628465 1486.1835867225166
0.6693667274191456


217512 83218
28 100
1638.6709779484695 1650.9100075797844
0.4823939025218336


28836 2555946
715 1074
1469.5265041760883 1467.1042109852608
0.5034859036396674


3475 4031
98 147
1584.7272800492985 1550.026540421

2088345 45571
631 374
1521.930748408173 1413.8642639671668
0.650691621068218


4047 54130
192 436
1513.8609293537036 1453.8642998681566
0.5854939699346361


51256 3586
689 690
1611.6772078466815 1483.5748770554849
0.6764306125120718


3607 3671
160 597
1434.7235854660235 1443.7083963271325
0.4870726993985947


176778 2526148
598 315
1525.3581646304347 1513.740367793247
0.5167131248517411


200399 3481
153 339
1611.029529733076 1494.240308170321
0.6620163655499313


3982 1570645
556 628
1623.6555216477545 1585.2305482317952
0.5550736210237411


2103686 2098986
440 696
1472.8427109543593 1482.608161131999
0.4859500871875332


76531 1554636
300 599
1574.1781123090202 1479.2281830407292
0.6333407697345164


802298 51259
373 320
1490.0883611368085 1429.3666460214322
0.5865065810637055


51288 200392
317 645
1496.0104570941858 1495.6117336526822
0.5005738089059883


167878 70414
650 271
1515.9168023873585 1480.7384808727293
0.5504533786724073


1601923 109210
494 679
1555.4808040744797 1479.

1863628 3850
385 327
1489.8768226741379 1563.7521118901632
0.39525868250797036


4487 53125
587 760
1483.0203584108012 1457.1696364842123
0.5371336804964847


1083703 169560
455 1104
1493.0807079635053 1483.7072025007233
0.5134862867757417


4515 3456
171 105
1634.5705193617312 1660.6804662469378
0.46249534306112333


58977 67412
592 487
1450.4582984959302 1495.8841125434628
0.43499697559366185


49884 51245
419 459
1495.9971392340797 1445.6248072531514
0.5719879197560326


1740216 373773
444 377
1615.0364626832265 1424.2578206143066
0.7499245901487863


27182 806774
214 244
1595.0981867876933 1455.7217934414386
0.6904694671754075


2594847 64428
551 648
1536.411131441877 1489.9876344295028
0.5664140080200829


10850 1398920
241 448
1525.0064982491572 1427.9376268613998
0.6361686674929451


51256 5164
689 510
1611.6772078466815 1506.1327323475753
0.6473846526503627


77442 55856
227 758
1542.2689239048782 1487.5435746382718
0.5781111922115457


1393071 191038
534 299
1506.92429574584 1

741 718
1490.4377830518615 1454.1381885609512
0.5520500643078939


171016 51330
746 1127
1530.5935102933543 1484.7040225341211
0.5656589122468609


191865 52950
762 1128
1517.3569544726406 1483.3000192898546
0.548855491191231


2044152 167844
439 660
1544.1814853563199 1463.2906164663557
0.6143525341912894


95375 51288
480 317
1532.666112951963 1496.0104570941858
0.5525568706178063


51790 200394
485 1129
1498.392315473217 1483.1508290488805
0.5219202025252841


69327 53292
476 407
1545.9619306285585 1528.661846313214
0.5248762665909661


162815 137424
316 346
1525.6807325370312 1510.8094656199758
0.521388413194529


734160 2092193
392 740
1514.5541572781403 1484.3135690725067
0.5434101370564769


4075 51795
356 1130
1517.8974310516817 1484.842189236881
0.5474273033081913


3746 1399808
298 183
1564.1490868904882 1500.255617580901
0.5909273647341752


252053 208113
488 443
1520.6863332456105 1462.0208516207354
0.5836330925626418


28298 5218
629 181
1559.5253799652041 1572.12799206026

In [12]:
utr_train[:25]

,index,resultid,winnerid,loserid,winnerset1,winnerset2,winnerset3,winnerset4,winnerset5,loserset1,loserset2,loserset3,loserset4,loserset5,prob
0,420,578,11063,45027,6,6,0,0,0,1,0,0,0,0,0.424898
1,2818,4026,847617,10788,1,6,6,0,0,6,3,4,0,0,0.507280
2,2816,4022,11076,2555950,6,6,0,0,0,3,2,0,0,0,0.616359
3,1593,2249,58235,79636,6,6,0,0,0,2,3,0,0,0,0.564893
4,1589,2244,3498,83218,4,6,6,0,0,6,4,4,0,0,0.295505
5,2082,2954,3691,3469,7,7,0,0,0,5,5,0,0,0,0.441408
6,3814,5486,3564,224142,6,7,6,0,0,4,5,1,0,0,0.510233
7,2083,2955,52337,51245,7,7,0,0,0,5,6,0,0,0,0.480530
8,3830,5505,3832,52294,6,6,7,0,0,1,1,6,0,0,0.462541
9,3832,5508,10788,1609206,6,5,7,0,0,2,7,5,0,0,0.509381


In [13]:
utr_test

,resultid,resultmonth,player1,player2,player1winprobability
0,3,2019-02,4372,3464,NaN
1,6,2019-02,3671,3507,NaN
2,8,2019-08,4584,53374,NaN
3,9,2019-08,10759,10751,NaN
4,13,2019-10,405461,1194009,NaN
...,...,...,...,...,...
1839,6524,2019-11,3654,87175,NaN
1840,6537,2019-11,79558,54130,NaN
1841,6543,2019-02,1517181,4031,NaN
1842,6545,2019-08,10751,10274,NaN


Forloop that will calculate the player 1 win probabilites in the test dataset.
Probabilities are calculated based on elo ratings of the two players that were obtained after looping 
through training set in chronological order. Elos are grabbed from **player_df** by using player id as foreign key reference

In [14]:
for i in range(0,len(utr_test)):
    m_winner = utr_test.iloc[i,2]
    m_loser = utr_test.iloc[i,3]
    
    winner_df_index = player_df.id[player_df.id == m_winner].index[0]
    loser_df_index = player_df.id[player_df.id == m_loser].index[0]
    
    winner_rating = player_df.iloc[winner_df_index,1]
    loser_rating = player_df.iloc[loser_df_index,1]
    
    current_prob = expectedValue(winner_rating, loser_rating)
    utr_test.at[i,'player1winprobability'] = current_prob

In [15]:
utr_test[:25]

,resultid,resultmonth,player1,player2,player1winprobability
0,3,2019-02,4372,3464,0.548170
1,6,2019-02,3671,3507,0.452522
2,8,2019-08,4584,53374,0.293999
3,9,2019-08,10759,10751,0.667230
4,13,2019-10,405461,1194009,0.557416
5,16,2019-10,52346,77442,0.512415
6,18,2019-03,10378,51673,0.739920
7,19,2019-11,3850,54846,0.634702
8,20,2019-07,207037,95506,0.473536
9,26,2019-09,10244,10839,0.462594


In [16]:
utr_test.to_csv(r'RileyMault.csv', index=False)